In [4]:
# Author: Harper Wang
# Title: OpenWeatherMap Weather Forecast

In [ ]:
# Install packages
import pprint
import requests
import sys
import numpy as np
import statistics as st
import pandas as pd

In [5]:
loc = ['Guilin,China',
  'Dissen,Germany',
  'Guatemala City,Guatemala',
    'Kandukur, India',
    'Nanaimo, British Columbia',
    'Uijeongbu-si, South Korea',
    'Yangon, Myanmar',
    'Jalpa de Mendez, Mexico',
    'Enugu, Nigeria',
    'Peterhead, Scotland',
    'Lima, Peru',
    'Singapore, Singapore',
    'Kaohsiung, Taiwan',
    'Grimesland, North Carolina',
  'Visalia,California',
  'Colonia del Sacramento,Uruguay']

In [6]:
api_key = 'f8647786d6d7d623966c4cd0e120b7dd'
    
#  Geolocate Peterhead, Scotland to get it's latitude and longitude 


URL_direct = 'http://api.openweathermap.org/geo/1.0/direct'
URL_forecast = 'http://api.openweathermap.org/data/2.5/forecast'

city_weather = []
for i in range(len(loc)):

    city = loc[i]
# there are many repeated city names in the world(limit#)
    geo = f'{URL_direct}?q={city}&limit=1&appid={api_key}'
    resp = requests.get( geo )

    if resp.status_code != 200:  # Failure?
        print( f'Error geocoding {city}: {resp.status_code}' )
        sys.exit( 1 )

    #  OpenWeatherMap returns a list of matching cities, up to the limit specified
      #  in the API call; even if you only ask for one city (limit=5), it's still
      #  returned as a 1-element list

    if len( resp.json() ) == 0:  # No such city?
        print( f'Error locating city {city}; {resp.status_code}' )
        sys.exit( 2 )

    json = resp.json()
    if type( json ) == list:  # List of cities?
        lat = json[ 0 ][ 'lat' ]
        lon = json[ 0 ][ 'lon' ]

    else:  # Unknown city?
        print( f'Error, invalid data returned for city {city}, {resp.status_code}' )
        sys.exit( 3 )

    #  Use Peterhead's latitude and longitude to get it's 5-day forecast in 3-hour
      #  blocks


    forecast = f'{URL_forecast}?lat={lat}&lon={lon}&appid={api_key}'
    resp = requests.get( forecast )

    if resp.status_code != 200:  # Failure?
        print( f'Error retrieving data: {resp.status_code}' )
        sys.exit( 4 )
    data = resp.json()

    i = 0
    max1_temp_list = []
    max2_temp_list = []
    max3_temp_list = []
    max4_temp_list = []

    for j in range(i, i+8):
        if data['list'][j]['dt_txt'].split(' ')[1] == '00:00:00':
            for k in range(i, i+8):
                max1_temp_list.append(data['list'][k]['main']['temp_max'])

            for k in range(i+8, i+16):
                max2_temp_list.append(data['list'][k]['main']['temp_max'])

            for k in range(i+16, i+24):
                max3_temp_list.append(data['list'][k]['main']['temp_max'])

            for k in range(i+24, i+32):
                max4_temp_list.append(data['list'][k]['main']['temp_max'])

    max_temp_all = []
    lists = [max1_temp_list, max2_temp_list, max3_temp_list, max4_temp_list]
    for temp_list in lists:
        max_temp = max(temp_list)
        max_temp_all.append(max_temp)

    avg_max_all = np.mean(max_temp_all)
    max_temp_all.append(avg_max_all)


    # for loop to creat 8 blocks' min tempa

    i = 0
    min1_temp_list = []
    min2_temp_list = []
    min3_temp_list = []
    min4_temp_list = []

    for j in range(i, i+8):
        if data['list'][j]['dt_txt'].split(' ')[1] == '00:00:00':
            for k in range(i, i+8):
                min1_temp_list.append(data['list'][k]['main']['temp_min'])

            for k in range(i+8, i+16):
                min2_temp_list.append(data['list'][k]['main']['temp_min'])

            for k in range(i+16, i+24):
                min3_temp_list.append(data['list'][k]['main']['temp_min'])

            for k in range(i+24, i+32):
                min4_temp_list.append(data['list'][k]['main']['temp_min'])

    min_temp_all = []
    lists = [min1_temp_list, min2_temp_list, min3_temp_list, min4_temp_list]

    for temp_list in lists:
        min_value = min(temp_list)
        min_temp_all.append(min_value)

    avg_min_all = np.mean(min_temp_all)

    min_temp_all.append(avg_min_all)


    all_temp = zip(min_temp_all, max_temp_all)
    result = list(all_temp)


    all=[]
    i = 0
    all.insert(0, city)
    for i in range(i, i+5):
        all.append('%.2f' % (min_temp_all[i] - 273.15))
        all.append('%.2f' % (max_temp_all[i] - 273.15))
    
    
    city_weather.append(all)
    
print(city_weather)

df = pd.DataFrame(city_weather)
header = ['City', 'Min 1', 'Max 1', 'Min 2', 'Max 2', 'Min 3', 'Max 3', 'Min 4', 'Max 4', 'Min Avg', 'Max Avg']
df.columns = header
df.to_csv("temp.csv", index=False)

# csv - given name
# column name
# "" or ''
# numbers with out ''

[['Guilin,China', '15.19', '22.48', '4.71', '23.32', '0.60', '3.97', '2.08', '2.20', '5.65', '12.99'], ['Dissen,Germany', '3.00', '5.75', '3.44', '6.18', '3.63', '6.10', '6.06', '7.93', '4.03', '6.49'], ['Guatemala City,Guatemala', '14.11', '21.70', '13.39', '22.12', '13.50', '23.04', '13.20', '21.97', '13.55', '22.21'], ['Kandukur, India', '21.70', '27.16', '21.61', '26.82', '21.58', '27.39', '22.19', '27.17', '21.77', '27.13'], ['Nanaimo, British Columbia', '5.69', '7.81', '6.09', '7.30', '6.50', '8.80', '5.77', '9.01', '6.01', '8.23'], ['Uijeongbu-si, South Korea', '5.95', '7.44', '2.57', '5.70', '-8.80', '-0.46', '-9.51', '-6.26', '-2.45', '1.61'], ['Yangon, Myanmar', '23.92', '33.60', '23.57', '33.39', '22.64', '33.11', '22.02', '33.01', '23.04', '33.28'], ['Jalpa de Mendez, Mexico', '20.82', '25.68', '21.05', '25.48', '20.83', '25.06', '18.67', '24.46', '20.34', '25.17'], ['Enugu, Nigeria', '21.59', '34.53', '21.95', '35.08', '22.06', '34.76', '22.78', '35.45', '22.10', '34.96'],